### Bond Duration
Macaulay duration is named after the economist Frederick Macaulay, who introduced the metric in 1938 as a way to measure the volatility of the price of a bond, or portfolio of bonds, to changes in interest rates. It is the weighted average time until the present value of the bond’s cash flows equals the current market price for the bond. Macaulay duration is measured in years and is often used by portfolio managers who implement bond immunization strategies. 

Modified duration was developed as a response to the sharp interest rate rises in the 1970s, as a method to assess a bonds sensitivity to interest rate changes. Modified duration 'modifies' Macaulay duration by multiplying it by 1 / (1 + (YTM / frequency).

The purpose of this notebook is to demonstate the calculation of a bonds Macaulay and Modified duration. For this purpose I encapsulate the calculations in a Bond class. Calculations from the Bond class are validated against a similar bond created using the Quantlib library.

In [1]:
class Bond():
    '''
    Bond class to demonstate calculation of Macaulay and Modified duration
    '''
    
    import numpy as np

    def __init__(self,spot_rates,terms,coupon_rate,face_value=100,frequency=2):     
        '''
        Initilise an instance of a Bond
        
        :param list spot_rates: List of rates to discount each payment
        :param list terms: List of terms before each payment is received, in years
        :param float coupon_rate: Coupon rate
        :param int face_value: Par value of the bond, defaults to 100
        :param int frequency: Frequency of payments, defaults to 2 - semiannual
        :return: void        
        '''
        
        self.spot_rates = spot_rates
        self.terms = terms
        self.coupon_rate = coupon_rate
        self.face_value = face_value
        self.frequency = frequency
        
        self.n_payments = len(self.terms)
        self.__set_spot_rates()
        
        
    @property
    def __discount_curve(self):
        if not hasattr(self, 'discount_curve'):
            self.discount_curve = []
            for i,t in enumerate(self.terms):
                self.discount_curve.append(1 / self.np.power( ( 1 + self.spot_rates[i]),t))
            self.discount_curve = self.np.array(self.discount_curve)

        return self.discount_curve
    
    
    @property
    def __cash_flow(self):
        if not hasattr(self, 'cash_flow'):
            self.cash_flow = []
            
            for i in range(self.n_payments):
                self.cash_flow.append(0.5 * self.coupon_rate * self.face_value)

                # add face to last payment
                if (i == self.n_payments - 1):
                    self.cash_flow[i] += self.face_value

        return self.cash_flow
    
    
    @property    
    def NPV(self):
        '''
        Net present value of cash flows. Clean Price of the bond.       
        '''
        
        if not hasattr(self, 'npv'):
            self.npv = self.np.sum(self.__cash_flow * self.__discount_curve)
            
        return self.npv 
    
    
    @property
    def macaulay_duration(self):
        '''
        Macaulay duration
        '''
        
        if not hasattr(self, 'macaulay'):
        
            discounted_cf = self.__cash_flow * self.__discount_curve

            # discounted cash flow weighted by term
            weighted_dcf = [discounted_cf[i] * term  for i,term in enumerate(terms)]


            self.macaulay = self.np.sum(weighted_dcf) / self.NPV

        return self.macaulay
    
    
    def modified_duration(self,ytm):
        '''
        Modified duration
        '''    
        
        return self.macaulay_duration / (1 + (ytm / self.frequency))
    
    
    def approx_ytm(self):
        '''
        Yeild to Maturity, approximated by ( Coupon + ((FV - PV ) /1 )) /( (FV + PV ) / Frequency)
        '''

        approx_ytm = ((self.coupon_rate * self.face_value) + ((self.face_value - self.NPV ) / 1 )) \
            / ((self.face_value + self.NPV ) / self.frequency)
        
        return round(approx_ytm, 4) 


    def __set_spot_rates(self):
        self.n_payments = len(self.terms)

        # spot_rates expects a list. if only one rate passed in then set all spot rates to that rate
        if len(self.spot_rates) == 1:
            for _ in range(1,n_payments):
                self.spot_rates.append(self.spot_rates[0])       

In [2]:
# bond attributes
spot_rates = [ 0.006, 0.006]
terms = [ 0.5, 1.0]
coupon_rate = .07

bond = Bond(spot_rates,terms,coupon_rate)

print ('NPV: ', round(bond.NPV,10))
print ('Macaulay Duration: ', round(bond.macaulay_duration,10))
print ('Approximate YTM: ', bond.approx_ytm())
print ('Modified Duration: ', round(bond.modified_duration(bond.approx_ytm()),10))

NPV:  106.3722507923
Macaulay Duration:  0.983597475
Approximate YTM:  0.0061
Modified Duration:  0.9806066248


### Validate the Bond class calculations
I validate the values returned from the Bond class by creating a fixed rate bond with the same attributes, using the Quantlib library.  I then compare the calculations from both the Bond class and the Quantlib library for NPV, Macaulay Duration, and Modified Duration. The following code is based on the Quantlib documentation.

In [3]:
# create an instance of bond using the Quantlib library
import QuantLib as ql

todaysDate = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = todaysDate
spotDates = [ql.Date(15, 1, 2015), ql.Date(15, 7, 2015), ql.Date(15, 1, 2016)]
spotRates = [0.06, 0.006, 0.006]

dayCount = ql.Thirty360()
calendar = ql.UnitedStates()
interpolation = ql.Linear()
compounding = ql.Compounded
compoundingFrequency = ql.Annual

spotCurve = ql.ZeroCurve(spotDates, spotRates, dayCount, calendar, interpolation,compounding, compoundingFrequency)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)


# define the bond.
issueDate = ql.Date(15, 1, 2015)
maturityDate = ql.Date(15, 1, 2016)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates()
businessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule (issueDate, maturityDate, tenor, calendar, businessConvention, businessConvention, dateGeneration, monthEnd)


# coupons
dayCount = ql.Thirty360()
couponRate = .07
coupons = [couponRate]

settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)


# create a bond engine with the term structure as input
# set the bond to use this bond engine
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
fixedRateBond.setPricingEngine(bondEngine)

In [4]:
# get the NPV using the Quantlib class
print('NPV: ', round(fixedRateBond.NPV(),10))

NPV:  106.3722507923


The calculation of the Bond class NPV was:

* NPV:  106.3722507923

The NPV of the Bond class matches the NPV of the Quantlib Bond class.

In [5]:
#calculate yield
targetPrice = fixedRateBond.cleanPrice()
day_count = dayCount
compounding = ql.Compounded
frequency = 2

ytm = fixedRateBond.bondYield(targetPrice, day_count, compounding, frequency)
print('YTM: ', round(ytm,10))

rate = ql.InterestRate(ytm, ql.Thirty360(), ql.Compounded, ql.Semiannual)

# calculate duration
macaulay_duration = ql.BondFunctions.duration(fixedRateBond,rate,ql.Duration.Macaulay)
print('Macaulay Duration: ', round(macaulay_duration,10))

modified_duration = ql.BondFunctions.duration(fixedRateBond,rate,ql.Duration.Modified)
print('Modified Duration: ', round(modified_duration,10))

YTM:  0.0059910219
Macaulay Duration:  0.983597475
Modified Duration:  0.9806598975


The calculations of the Bond class were:

* Approximate YTM:  0.0061
* Macaulay Duration:  0.983597475
* Modified Duration:  0.9806066248

The Macaulay Duration is the same for both the Bond class and the Quantlib class. The Modified Duration matches the Quantlib value to 4 decimal places. It is not exact, as the approximate YTM was used in the Bond class calculation. Below I pass in the Quantlib YTM, and the Bond class Modified Duration matches the Quantlib value exactly.

In [6]:
# pass in the yeild to maturity calculated by the Quantlib class 
print ('Modified Duration using Quantlib ytm: ', round(bond.modified_duration(ytm),10))

Modified Duration using Quantlib ytm:  0.9806598975
